In [5]:
import numpy as np
import matplotlib.pyplot as plt

_beta = 1.5
_sigma = 0.6966
_eps = 1e-8


def _levy(dim, n):
    r1 = np.random.normal(size=(n, dim))
    r2 = np.random.normal(size=(n, dim))
    return 0.01 * ((r1 * _sigma) / np.power(np.abs(r2), 1.0 / _beta))


def _variable_param(i, maxi, agents=1):
    w = 0.9 - i * ((0.9 - 0.4) / maxi)
    my_c = 0.1 - i * ((0.1 - (-0.1)) / maxi)
    my_c = 0 if my_c < 0 else my_c
    s = 2 * np.random.sample((agents, 1)) * my_c  # Seperation weight
    a = 2 * np.random.sample((agents, 1)) * my_c  # Alignment weight
    c = 2 * np.random.sample((agents, 1)) * my_c  # Cohesion weight
    f = 2 * np.random.sample((agents, 1))  # Food attraction weight
    e = my_c  # Enemy distraction weight
    return a, c, e, f, s, w


def variable_plot(param_fun, maxi, n):
    iter_x = np.arange(maxi)
    arr = np.zeros((maxi, 6))
    for i in range(maxi):
        res = np.zeros((n, 6))
        for j in range(n):
            res[j, :] = np.asarray(param_fun(i, maxi, 1))
        arr[i, :] = np.mean(res, axis=0)
    plt.plot(iter_x, arr[:, 0], label="a")
    plt.plot(iter_x, arr[:, 1], label="c")
    plt.plot(iter_x, arr[:, 2], label="e")
    plt.plot(iter_x, arr[:, 3], label="f")
    plt.plot(iter_x, arr[:, 4], label="s")
    plt.plot(iter_x, arr[:, 5], label="w")
    plt.title("Zbieznosc parametrow")
    plt.xlabel("Liczba iteracji")
    plt.ylabel("Wartosc wagi")
    plt.legend(fontsize='medium')
    plt.savefig("paramevolution.png")
    plt.show()


def _get_radius(i, maxi, lbd, ubd):
    return (ubd - lbd) * (0.25 + ((2.0 * i)/maxi))


def _random_population(cols,choices=[-1,0,1,2],rows=12 ):
    #return a n*lbd(row*col) size matrix with every element in [0,ubd)
    # will need to change to discrete
    # return np.random.random((n, lbd.size)) * (ubd - lbd) + lbd
    return np.random.choice(choices,(rows, cols))

def _get_neighbours_matrix(pos, radius, agents):
    t = np.abs(pos - pos[:, np.newaxis]) < radius
    return np.all(t, 2) - np.eye(agents, dtype=np.int8)


def _get_neighbours_vector(pos, radius, v):
    t = np.abs(pos - v) < radius
    return np.all(t, 1) + 0.0


def _divide(l, m, default):
    m2 = np.repeat(m, l.shape[1]).reshape(l.shape)
    ind_non0 = np.where(m2 > 0)
    ind_eq0 = np.where(m2 == 1)
    l[ind_non0] /= m2[ind_non0]
    l[ind_eq0] = default[ind_eq0]
    return l


def _border_reflection(pos, lbd, ubd):
    diff = ubd - lbd
    f = np.floor(pos/diff - lbd/diff)
    lm = (np.mod(f, 2.0) == 1.0).real * (ubd + lbd)
    pos = (pos - diff * f) * np.power(-1.0, f) + lm
    return pos


In [6]:
k =_random_population(14,choices=[-1,0,1,2],rows=12 )
k


array([[ 1,  1, -1,  2,  2,  0, -1, -1,  0, -1,  0, -1,  2,  1],
       [ 1,  2,  1,  1,  2,  2,  2,  1, -1,  0,  2,  2,  1, -1],
       [ 1,  0,  1,  0, -1,  0,  1,  0,  0, -1, -1,  1, -1,  2],
       [-1,  1,  2,  0,  2,  1, -1,  0,  0, -1,  0, -1,  1,  0],
       [ 2, -1, -1,  0,  2,  2,  0, -1, -1, -1,  1,  0,  1,  1],
       [ 0, -1,  0,  1,  2, -1, -1,  1,  2,  2,  2,  0, -1,  1],
       [ 0,  1,  1,  1,  0, -1, -1,  0,  1, -1, -1,  0,  0,  1],
       [ 1, -1, -1,  1,  0, -1,  0,  2,  1, -1,  2,  0,  2,  1],
       [-1,  1,  2, -1,  0,  0,  0,  0, -1,  2,  1,  0,  2,  1],
       [-1, -1, -1,  0, -1, -1,  1,  1, -1, -1, -1,  2,  2,  1],
       [ 1,  0, -1,  2,  1,  1,  0,  1, -1,  0,  0,  2, -1,  0],
       [ 1,  0,  0,  0,  0,  1,  2,  2, -1,  0, -1,  1,  2,  2]])